# daum 페이지의 뉴스 기사를 읽어 형태소 분석 후 word2vec을 적용
####<단어 간 유사도 확인하기>


In [ ]:
!pip install konlpy  #konlpy는 자바로 만들어졌으므로 자바가 설치되어있어야 함

In [ ]:
from konlpy.tag import Okt
import pandas as pd

okt= Okt()

with open('news.txt',mode='r') as obj:
    lines=obj.read().split('\n')

print(lines)

In [ ]:
# 명사만 추출해 단어 빈도 수를 확인
wordDic= {}

for line in lines:
    datas= okt.pos(line) # 품사태깅 - 단어마다 품사가 옆에 따라 붙는 것
    # print(datas)
    for word in datas:
        if word[1]=='Noun':
            # print(word[0])
            if len(word[0]) >=2:
                if not(word[0] in wordDic):
                    wordDic[word[0]] = 1
                wordDic[word[0]]+=1

print(wordDic) # counting을 해서 이 기사에 대한 정보를 알수 있다.

keys = sorted(wordDic.items(), key=lambda x:x[1], reverse=True)
print(keys)
print()

# keys 자료를 DataFrame에 담기
wordList = []
countList= []

for word, count in keys[:20]:
    wordList.append(word)
    countList.append(count)

df=pd.DataFrame()
df['word'] = wordList
df['count'] = countList

print(df.head(3))

# pandas의 기능으로 기술통계 작업 처리를 해주면 됨...

In [ ]:
# 단어 간 유사도 구하기
results= [] #한글만 기억

with open('news.txt',mode='r') as obj:
    lines=obj.read().split('\n')

    for line in lines:
        datas = okt.pos(line, stem=True) # stemming - 동사나 형용사에 대해서 원형 어근으로 출력, 한가한 -> 한가하다
        print(datas)
        imsi = []
        for word in datas:
            if not word[1] in ['Number','Josa','Punctuation','Alpha','Modifier','Suffix','Foregin']:
                if len(word[0]) >= 2:
                    imsi.append(word[0])
        imsi2 = (' '.join(imsi)).strip()
        results.append(imsi2)


In [ ]:
print(results)

fileName ='daumnews.txt'
with open(fileName, mode='w') as obj:
    obj.write('\n'.join(results))

from gensim.models import word2vec
# 코사인 유사도를 이용해 단어와 단어사이 수치를 알려줌

lineObj = word2vec.LineSentence(fileName)

model = word2vec.Word2Vec(sentences=lineObj, vector_size=100, window=10, min_count=1, sg=1)# 2보다 작은 녀석은 제외
# sg=0: CBOW(주변단어로 중심단어를 예측), sg=1: Skip-Gram(중심단어로 주변단어를 예측)
# 주변 단어를 몇개를 창조하는 것은 윈도우로 결정함
# 100차원의 벡터를 만듦. 주변단어를 예측하는 알고리즘을 이용해서 모델이 학습을 함
print(model)

# model.init_sims(replace=True) # 필요없는 메모리 해제

# positive : 단어 사전에 해당 단어가 있을 확률, 가까운 단어를 찾음
# negative : 단어 사전에 해당 단어가 없을 확률, 먼 단어를 찾음
print(model.wv.most_similar(positive=['아메리카노'])) #수출이란 단어와 유사도에서 긍정적 기여 목록
print(model.wv.most_similar(negative=['시키다'])) #수출이란 단어와 유사도에서 부정 기여 목록
print(model.wv.most_similar(negative=['분비'], topn=5))  #기여도가 높은 녀석 다섯개만 나온다.


희소표현

밀집벡터 - 공간을 내 마음대로 데이터들을 표현할 수 있는 것
데이터들을 0에서부터 1사이에 가둘수 있는 것
차원을 내가 정해주고 그 차원내에서 수치들이 표현될 수 있도록 하는 것
